# Inverse Compton Scattering

At the low energy limit, Tracy calculated an approximate expression for dN/(dt deps1), which is the number of photons per scattered energy eps1 per unit time. This expression is presented as an integrand, integrated over the CMB spectrum.

In [4]:
import numpy as np

import darkhistory.physics as phys

In [19]:
def loweng_integrand(eleckineng, photeng, CMBeng):
    """
    
    Parameters
    ----------
    
    eleckineng : float
        Kinetic energy of the electron.
    photeng : ndarray
        Energy of the scattered photon. 
    CMBeng : float
        Energy of the CMB photon (to be integrated over)
    """
    gamma = 1 + eleckineng/phys.me
    beta = np.sqrt(1 - 1/(gamma**2))
    
    def prefac(CMBeng): 
        return phys.c*(3/16)*(
            phys.thomson_xsec/((gamma**3)*(beta**2)*(CMBeng**2))
        )
    
    def integrand(CMBeng, photeng):
        """
        CMBeng is a double, photeng can be an ndarray.
        """
    
        photenghigh = (
            CMBeng*(1+beta**2)/(beta**2)*np.sqrt((1+beta)/(1-beta))
            + (2/beta)*np.sqrt((1-beta)/(1+beta))*photeng
            - ((1-beta)**2)/(beta**2)*np.sqrt((1+beta)/(1-beta))*(
                 photeng**2/CMBeng
            )
            + 2/(gamma*beta**2)*photeng*np.log(
                 (1-beta)/(1+beta)*photeng/CMBeng
            )
        )
        
        photenglow = (
            - CMBeng*(1+beta**2)/(beta**2)*np.sqrt((1-beta)/(1+beta))
            + (2/beta)*np.sqrt((1+beta)/(1-beta))*photeng
            + (1+beta)/(gamma*beta**2)*(photeng**2/CMBeng)
            - 2/(gamma*beta**2)*photeng*np.log(
                 (1+beta)/(1-beta)*photeng/CMBeng
             )
        )
        
        return np.where(photeng > CMBeng, photenghigh, photenglow)
        
    return prefac(CMBeng)*integrand(CMBeng, photeng)    
    
    

In [20]:
testeleceng = 1e6
testphoteng = np.array([1e-1, 1, 1e1, 1e2])
loweng_integrand(testeleceng, testphoteng, 3)

array([  1.85587700e-18,   1.72426631e-16,   4.01365671e-16,
        -7.47769962e-18])

In [7]:
a = np.array([1,2,3,4,5])
a > 3

array([False, False, False,  True,  True], dtype=bool)

In [18]:
3.67800/2.69290811

1.365809693372716